In [ ]:
from pathlib import Path
import os

os.environ["KERAS_BACKEND"] = "jax"

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    f1_score,
    precision_recall_fscore_support,
    roc_auc_score,
    roc_curve,
    auc,
    precision_recall_curve,
    confusion_matrix
)
import jax
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from var import DATA_OUT, IMAGE_OUT, FORECAST_HOURS_IN_ADVANCE

In [ ]:
df = pd.read_pickle(Path(DATA_OUT, 'df_dataset.pickle'))

In [ ]:
X = df[
    [
        'ie_fix',
        'ie_mav_6h',
        'iu_fix',
        'iu_mav_6h',
        'hf',
        'hf_mav_2h',
        'f_107_adj',
        'hp_30',
        'smr',
        'solar_zenith_angle',
    ]
].copy()

y = df[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].copy()

In [ ]:
# model = Sequential(
#     [
#         LSTM(
#             64,
#             input_shape=(X.shape[0], X.shape[1]),
#         ),
#         Dense(1, activation='sigmoid'),
#     ],
# )

In [ ]:
# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['f1'])

In [ ]:
# batch_size = 32
# epochs = 10
# 
# model.fit(
#     X,
#     y,
#     batch_size=batch_size, 
#     epochs=epochs,
# )

In [ ]:
# Valutazione del modello
loss, accuracy = model.evaluate(inputs, targets)
print("Loss:", loss)
print("Accuracy:", accuracy)

In [ ]:
X_ = np.reshape(
    X,
    (X.shape[0], 1, X.shape[1])
)

In [ ]:
X_.shape

In [ ]:
inputs = Input(
    shape=(X_.shape[1], X_.shape[2])
)

lstm = LSTM(
    units=64, activation='relu'
)(inputs)

outputs = Dense(
    units=1, activation='sigmoid'
)(lstm)

model = Model(
    inputs=lstm, outputs=outputs
)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['f1'],
)

In [ ]:
# Addestra il modello
model.fit(X_, y, epochs=100, batch_size=32)

In [ ]:
# Valuta il modello (opzionale)
loss = model.evaluate(X_test, y_test)